# Developing for Intake

Intake has been designed with a plugable structure, to allow adding new data sources and other features as easily as possible. Here we demonstrate making a new plugin to read data from the [GitHub API](https://developer.github.com/v3/), using a small and handy package that already exists, `pygithub`. We will access only one particular part of the API, as a simple example.

The directory `intake-github` contains a full package to do the job.

In [ ]:
cd intake-github/

The contents are typical python package files:

In [ ]:
!find .

The `setup.py` and `meta.yaml` are standard distutils and conda package information boilerplate, specifying dependencies and a little metadata. The `LICENSE` file is empty, but in general, all open-source projects should have an appropriate license.

This example contains just one DataSource class, although it could have had several related source.

The [source code](intake-github/intake_github/__init__.py) has very little code: a subclass of `DataSource` with the following class-level attributes:
```python
    container = 'dataframe'
    version = '0.0.1'
    partition_access = False
    name = 'github_issues'
```
These name the new plugin, give it a version and an output data type (Pandas dataframe), and specify that the data will always be loaded in a single shot, without partitioning. Whether or not a dataframe is the right respesentation for this data is left for another discussion.

The only real logic is in the read function:
```python
    def _get_partition(self, _):
        from github import Github
        import pandas as pd
        gh = Github()
        repo = gh.get_repo('%s/%s' % (self.organization, self.repo))

        raw_data = repo.get_issues(**self.ghkwargs)
        data = {}
        for column in ['number', 'title', 'user', 'state', 'comments',
                            'created_at', 'updated_at', 'body']:
            # user is special case
            if column == 'user':
                data['user'] = [issue.user.login for issue in raw_data]
            else:
                data[column] = [getattr(issue, column) for issue in raw_data]

        return pd.DataFrame(data)
```
This code is typical of the kind of coersion of data that might be required for different sources. Since we do not attempt partitioning here, the `read()` method is guaranteed to do the right thing.
The external module `pygithub` takes care of actually formatting the HTTP REST calls to the github API. We simply pass on keyword arguments, so if a catalog author would like more fine control over what is downloaded for any specific instance of this data source, they would have to read the `pygithub` and/or GitHub API documentation.

We can install the package with `pip`, or we could build the conda package and install that instead. To distribute the package, it could be uploaded to PyPI, to a user conda channel on anaconda.org, or added as a feedstock to conda-forge.

In [ ]:
# install with dependencies. Suppress text output. Can take a little while.
!pip install -e . > /dev/null

Since the class is in the top-level of the package, and the package name starts with `intake_`, it will be scanned when Intake is imported.
Now the plugin automatically appears in the set of known plugins in the Intake registry, and an associated `intake.open_github_issues` function is created at import time.

In [ ]:
import intake

In [ ]:
'github_issues' in intake.registry


If this automatic behaviour is not desired, then either catalogs would need to reference the plugin module/location explicitly in the `plugins:` section, or the package would need to insert its class into the Intake registry when *it* is imported.

Now we can use our new plugin to load data.

In [ ]:
source = intake.open_github_issues('ContinuumIO', 'intake')

In [ ]:
source.discover()

In [ ]:
out = source.read()

In [ ]:
# currently open issues and PRs in the Intake repo
out[['number', 'title', 'state']]

The **point** of doing this, is to enable data engineers to specify the plugin in their catalogs or when distributing data packages with conda. Notice that in YAML form, our data source references the name that we gave to the plugin in the source. If this plugin were not found on the system when such a catalog entry is loaded, an error would be raised. The second form also includes the optional plugin section.

In [ ]:
print(source.yaml())

In [ ]:
print(source.yaml(True))

### Summary

In a small amount of code, we have shown how to wrap a third-party data-loading package, to make it available to Intake as a plugin. The creation of the plugin enables inclusion of data-sets made from this kind of data possible in data catalogs. In this way, we build a uniform interface to access a wide variety of arbirtary data sources.